# 图片：背景为黑色，有很多白色填充的。

# 下面代码是去除图片中自定义面积小的轮廓， 将大的轮廓填充为白色

In [ ]:
import cv2
 
imgfile = "IMG_3200.png"
img = cv2.imread(imgfile)
h, w, _ = img.shape
 
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
 
# Find Contour
_, contours, hierarchy = cv2.findContours( thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
 
# 需要搞一个list给cv2.drawContours()才行！！！！！
c_max = []
for i in range(len(contours)):
    cnt = contours[i]
    area = cv2.contourArea(cnt)
 
    # 处理掉小的轮廓区域，这个区域的大小自己定义。
    if(area < (h/10*w/10)):
        c_min = []
        c_min.append(cnt)
        # thickness不为-1时，表示画轮廓线，thickness的值表示线的宽度。
        cv2.drawContours(img, c_min, -1, (0,0,0), thickness=-1)
        continue
    #
    c_max.append(cnt)
    
cv2.drawContours(img, c_max, -1, (255, 255, 255), thickness=-1)
 
cv2.imwrite("mask.png", img)
cv2.imshow('mask',img)
cv2.waitKey(0)


# 下面代码是只保留最大轮廓，填充为白色。

In [ ]:
# coding = utf-8
 
import cv2    
    
imgfile = "0_mask.png"    
img = cv2.imread(imgfile)    
h, w, _ = img.shape    
    
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    
    
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)    
    
# Find Contour    
_, contours, hierarchy = cv2.findContours( thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)    
    
# 需要搞一个list给cv2.drawContours()才行！！！！！    
c_max = []  
max_area = 0  
max_cnt = 0  
for i in range(len(contours)):  
    cnt = contours[i]  
    area = cv2.contourArea(cnt)  
    # find max countour  
    if (area>max_area):  
        if(max_area!=0):  
            c_min = []  
            c_min.append(max_cnt)  
            cv2.drawContours(img, c_min, -1, (0,0,0), cv2.FILLED)  
        max_area = area  
        max_cnt = cnt  
    else:  
        c_min = []  
        c_min.append(cnt)  
        cv2.drawContours(img, c_min, -1, (0,0,0), cv2.FILLED)  
  
c_max.append(max_cnt)  
  
  
cv2.drawContours(img, c_max, -1, (255, 255, 255), thickness=-1)    
    
cv2.imwrite("mask.png", img)    
cv2.imshow('mask',img)    
cv2.waitKey(0)    


# 进一步获取外接矩形

In [ ]:
import cv2
import numpy as np
 
img = cv2.pyrDown(cv2.imread("hammer.jpg ", cv2.IMREAD_UNCHANGED))
 
ret, thresh = cv2.threshold(cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY) , 127, 255, cv2.THRESH_BINARY)
image, contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
for c in contours:
  # find bounding box coordinates
  x,y,w,h = cv2.boundingRect(c)
  cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)
 
  # find minimum area
  rect = cv2.minAreaRect(c)
  # calculate coordinates of the minimum area rectangle
  box = cv2.boxPoints(rect)
  # normalize coordinates to integers
  box = np.int0(box)
  # draw contours
  cv2.drawContours(img, [box], 0, (0,0, 255), 3)
  
  # calculate center and radius of minimum enclosing circle
  (x,y),radius = cv2.minEnclosingCircle(c)
  # cast to integers
  center = (int(x),int(y))
  radius = int(radius)
  # draw the circle
  img = cv2.circle(img,center,radius,(0,255,0),2)
 
cv2.drawContours(img, contours, -1, (255, 0, 0), 1)
cv2.imshow("contours ", img)
 
cv2.waitKey()
cv2.destroyAllWindows()


# 根据mask获取原图的矩形区域

In [ ]:
# coding = utf-8
 
import cv2    
  
def get_crop_img(img_mask, img_org):
    gray = cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY)    
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)    
    # Find Contour    
    _, contours, hierarchy = cv2.findContours( thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)       
    x1_min=0
    y1_min=0
    x2_max=0
    y2_max=0
    j=0
    for i in range(len(contours)):  
        cnt = contours[i]
        area = cv2.contourArea(cnt)
        if area < 90000:
            continue
        j = j+1
        x,y,w,h = cv2.boundingRect(cnt)
        if j==1:
            x1_min, y1_min, x2_max, y2_max = x,y,x+w,y+h
        if x < x1_min:
            x1_min = x
        if y < y1_min:
            y1_min = y
        if x+w > x2_max:
            x2_max = x+w
        if y+h > y2_max:
            y2_max = y+h
    img_crop = img_org[y1_min:y2_max,x1_min:x2_max,:]
    return img_crop
 
# cv2.rectangle(img_org, (x1_min,y1_min), (x2_max, y2_max), (0, 255, 0), 2)  
 
 
img_file = "0.jpg"
imgfile_seg = "0_mask.png"    
img_org = cv2.imread(img_file) 
img_mask = cv2.imread(imgfile_seg)   
h, w, _ = img_mask.shape      
img_crop = get_crop_img(img_mask, img_org)
cv2.imwrite("mask.png", img_crop)    
# cv2.imshow('mask',img)    
# cv2.waitKey(0)   
